<a href="https://colab.research.google.com/github/MNabildawami/deteksiface/blob/main/deteksiface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install pyheif


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 21.5 MB/s eta 0:00:00


In [15]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
import cv2
import dlib
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pyheif
from PIL import Image
import io

# Unduh dan ekstrak model landmark wajah
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

# Muat model detektor wajah dlib
detector = dlib.get_frontal_face_detector()

# Muat model prediktor landmark wajah (pre-trained model)
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Fungsi untuk mengekstrak landmark dari wajah dan mengembalikan sebagai list data
def extract_face_landmarks_data(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # Periksa apakah wajah terdeteksi
    if len(faces) == 0:
        print("Tidak ada wajah yang terdeteksi.")
        return None

    landmarks_data = []  # List untuk menyimpan data landmark dari satu gambar

    for face in faces:
        landmarks = predictor(gray, face)
        # Loop melalui 68 titik landmark
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmarks_data.append((x, y))  # Tambahkan koordinat (x, y) ke list
    return landmarks_data

# Fungsi untuk membaca gambar HEIC dan mengkonversinya menjadi array NumPy
def read_heic_image(heic_file):
    heif_file = pyheif.read(heic_file)
    image = Image.open(io.BytesIO(heif_file.data))
    return np.array(image)

# Fungsi untuk memproses semua gambar dalam folder dan mengembalikan dataframe dengan landmark
def process_folder_images(folder_path):
    data = []
    labels = []  # Menyimpan label untuk setiap gambar

    # Loop untuk setiap file gambar di folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Cek apakah file berformat HEIC atau format gambar lain
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.heic')):  # Filter hanya file gambar
            # Jika gambar dalam format HEIC
            if file_name.lower().endswith('.heic'):
                image = read_heic_image(file_path)
            else:
                image = cv2.imread(file_path)

            # Cek apakah gambar berhasil dimuat
            if image is None:
                print(f"Gambar {file_name} tidak ditemukan atau tidak bisa dimuat!")
                continue

            # Ekstraksi fitur geometri sebagai data
            landmarks_data = extract_face_landmarks_data(image)

            if landmarks_data is not None:
                # Tambahkan hasil ekstraksi ke list data, termasuk nama file
                flattened_landmarks = [coord for point in landmarks_data for coord in point]  # Flatten koordinat
                data.append(flattened_landmarks)

                # Asumsi bahwa label berdasarkan folder atau nama file
                label = file_name.split('_')[0]  # Menetapkan label berdasarkan bagian nama file
                labels.append(label)

    # Periksa apakah ada lebih dari satu kelas di label
    if len(np.unique(labels)) < 2:
        raise ValueError("Data latih hanya memiliki satu kelas. Harap pastikan ada lebih dari satu kelas.")

    # Buat kolom untuk dataframe
    column_names = [f'x{i+1}' for i in range(68)] + [f'y{i+1}' for i in range(68)]

    # Buat dataframe dari data
    df = pd.DataFrame(data, columns=column_names)

    return df, labels

# Tentukan folder yang berisi gambar
folder_path = '/content/drive/MyDrive/footage item pengpol /nabil'  # Ganti dengan path folder Anda

# Proses semua gambar di folder dan simpan hasil ekstraksi ke dataframe
landmarks_df, labels = process_folder_images(folder_path)

# Membagi data latih dan data uji (70% data latih dan 30% data uji)
X_train, X_test, y_train, y_test = train_test_split(landmarks_df, labels, test_size=0.3, random_state=42)

# Latih model SVM dengan data latih
svm_classifier = SVC(kernel='linear')  # Gunakan kernel linear untuk SVM
svm_classifier.fit(X_train, y_train)

# Prediksi label untuk data uji
y_pred = svm_classifier.predict(X_test)

# Hitung akurasi
accuracy = accuracy_score(y_test, y_pred)

# Tampilkan hasil pengujian dan akurasi
print(f"Akurasi model SVM: {accuracy * 100:.2f}%")

# Simpan hasil ekstraksi ke file CSV
output_csv = 'landmarks_output.csv'
landmarks_df.to_csv(output_csv, index=True)

# Simpan hasil ekstraksi ke file Excel
output_excel = 'landmarks_output.xlsx'
landmarks_df.to_excel(output_excel, index=True)  # Simpan ke Excel

# Tampilkan pesan sukses
print(f"Data landmark berhasil disimpan dalam file Excel: {output_excel}")


--2024-12-12 19:32:13--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.12’

shape_predictor_68_ 100%[===================>]  61.07M  1.64MB/s    in 20s     

2024-12-12 19:32:33 (3.09 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.12’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.
Akurasi model SVM: 100.00%
Data landmark berhasil disimpan dalam file Excel: landmarks_output.xlsx
